# Deriving Weights of Simple Linear Elasticity Model using ML

In [0]:
%pip install linearmodels

In [0]:
dbutils.library.restartPython()

## 1. Finidng Routetype Clusters

In [0]:
# Import Libraries
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.functions import weekofyear
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
from pyspark.sql.functions import substring, col
from datetime import datetime

# Import Data

sales_history = spark.read.table('data_experience_commercial.cbt_1423_rtsuite.master_uat').select('flightkey', F.col('charge_dt').cast('date'), 'unt_net', 'chargeproduct', 'dtg')
dimensions_history_ = spark.read.table('data_experience_commercial.cbt_0923_segmentfinder.dimensions_history').select('flightkey', 'onsale_dt', 'ty_capacity', F.col('flight_dt').cast('date'))
dimensions_history = dimensions_history_.withColumn('sectormonth', F.concat(substring(col('flightkey'), 9, 6), substring(col('flightkey'), 5, 2)))

# Filter Data

filtered_sales = sales_history.filter((F.col('chargeproduct')=='Ticket') & (F.col('dtg')>=0)) # ticket only + eliminate covid, ss, and not yet flown
filtered_dimensions = dimensions_history.filter((F.datediff(F.col('flight_dt'), F.col('onsale_dt')) >= 168) & (F.col('flight_dt') >= '2023-04-01') & (F.col('flight_dt') <= '2024-03-31')) # eliminate late top-ups, filter for LY flight dates 

# Preprocessing - filling missing charge dates

dsh = filtered_sales.join(filtered_dimensions, on='flightkey', how='inner') # join tables
dshsmooth = dsh.groupby('flightkey','charge_dt').agg(F.sum('unt_net').alias('unt_net'), F.first('onsale_dt').alias('onsale_dt'), F.first('flight_dt').alias('flight_dt')) # aggregate into daily flight sales
date_range = dshsmooth.groupBy('flightkey').agg(F.min('onsale_dt').alias('start_date'), F.least(F.first('flight_dt'), F.lit(datetime.now().date())).alias('end_date')) # define flight onsale period
index = date_range.withColumn('charge_dt_ts', F.explode(F.sequence(F.col('start_date'), F.col('end_date')))).withColumn('charge_dt', F.col('charge_dt_ts').cast('date')) # create index of dates between onsale and flight date
dshjoin = index.join(dshsmooth, on=['flightkey', 'charge_dt'], how='left').drop('start_date', 'end_date','onsale_dt','flight_dt','charge_dt_ts').fillna(0) # join index with daily sales
window_spec = Window.partitionBy('flightkey').orderBy(F.col('charge_dt')) # create window for rolling pax sum
dsh_pax = dshjoin.withColumn('pax_net', F.sum('unt_net').over(window_spec)) # calculate current pax sum - currently neglects cancellations

# Preprocessing - creating LF by dtg progress remaining curves for each sector week

final_dsh = dsh_pax.join(filtered_dimensions, on='flightkey', how='left').drop('unt_net') # join daily sales with dimensions
curves = final_dsh.withColumn('total_booking_days', F.datediff(F.col('flight_dt'), F.col('onsale_dt'))) # calculate on sale period length in days
curves = curves.withColumn('dtg', F.datediff(F.col('flight_dt'), F.col('charge_dt'))) # calculate dtg
normal_curves = curves.withColumn('dtg_pr', F.col('dtg') / F.col('total_booking_days')) # express dtg progress remaining as dtg as a fraction of total booking days
normal_curve_buckets = normal_curves.withColumn('dtg_bucket', (F.floor(F.col('dtg_pr') * 100)).cast('int'))  # split dtg_pr into percentile buckets
aggregated_normal_curves = normal_curve_buckets.groupby('sectormonth', 'dtg_bucket').agg(F.sum('ty_capacity').alias('ty_capacity'), F.sum('pax_net').alias('pax_net')).orderBy('dtg_bucket') # aggregate by sector week and dtg bucket
df = aggregated_normal_curves.withColumn('load_factor', F.col('pax_net')/F.col('ty_capacity')).drop('ty_capacity', 'pax_net').toPandas() # create pandas dataframe of LF by dtg progress remaining by sector week

# storing original dataframe

df.info()
df_original = df.copy()
df = df_original.copy()

In [0]:
df_pivot = df.pivot_table(index='sectormonth', columns='dtg_bucket', values='load_factor', aggfunc='mean').fillna(0)
df_cluster = df_pivot[df_pivot[1] > 0]
df_cluster.head()

In [0]:
# Removing all routes that didnt have final LF > 60%

df_sold = df_cluster[df_cluster[1] > 0.6]
df_sold.info()

In [0]:
# Expressing Load Factor as a function of final LF
df_sold = df_sold.div(df_sold[1], axis=0)

In [0]:
# initial LF set to 0

row_min = df_sold.min(axis=1)
row_max = df_sold.max(axis=1)

df_sold = df_sold.subtract(row_min, axis=0).divide(row_max - row_min, axis=0)
df_sold.head()

In [0]:
import mlflow

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
k_range = range(2, 11)
wcss = []
for i in k_range: 
    kmeans = KMeans(n_clusters = i, init = 'k-means++', n_init=10)
    kmeans.fit(df_sold)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(k_range, wcss, marker='o', linestyle='--')
plt.title('Elbow Method')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('WCSS (Inertia)')
plt.xticks(k_range) 
plt.grid(True)
plt.show()

In [0]:
df_sold_original = df_sold.copy()

In [0]:
df_sold = df_sold_original.copy()

In [0]:
kmeans = KMeans(n_clusters = 5, init = 'k-means++', n_init=10)
kmeans.fit(df_sold)
df_sold['cluster_label'] = kmeans.labels_   
curves_clusters = df_sold.groupby('cluster_label').mean()

curves_clusters.T.plot(figsize=(20, 8))

plt.xlabel('% DTG remaining')
plt.ylabel('LF / LF_final')
plt.title('Isolated booking curve shape clustering "Banana split"')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(title=' # Cluster')
plt.show()

In [0]:
col40 = df_sold.groupby('cluster_label')[40].mean()
sorted_clusters = col40.sort_values()

auto_mapping = {}
for new_rank, old_label in enumerate(sorted_clusters.index, start=1):
    auto_mapping[old_label] = new_rank

df_sold['new_cluster_label'] = df_sold['cluster_label'].map(auto_mapping)

new_curves_clusters = df_sold.drop(columns=['cluster_label']).groupby('new_cluster_label').mean()

new_curves_clusters.T.plot(figsize=(20, 8))

plt.xlabel('% DTG remaining')
plt.ylabel('LF / LF_final')
plt.title('Isolated booking curve shape clustering "Banana split"')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(title=' # Cluster')
plt.show()

In [0]:
df_sold['cluster_label'].value_counts()

In [0]:
FY24Q3 = pd.read_csv('/Workspace/Users/barney.hodge@easyjet.com/FY24Q3.csv')
FY24Q3.sort_values(by='flight_time', inplace=True)
FY24Q3.info()

In [0]:
FY24Q4 = pd.read_csv('/Workspace/Users/barney.hodge@easyjet.com/FY24Q4.csv')
FY24Q4.sort_values(by='flight_time', inplace=True)
FY24Q4.info()

In [0]:
FY25Q1 = pd.read_csv('/Workspace/Users/barney.hodge@easyjet.com/FY25Q1.csv')
FY25Q1.sort_values(by='flight_time', inplace=True)
FY25Q1.info()

In [0]:
FY25Q2 = pd.read_csv('/Workspace/Users/barney.hodge@easyjet.com/FY25Q2.csv')
FY25Q2.sort_values(by='flight_time', inplace=True)
FY25Q2.info()

In [0]:
FY25Q3_1 = pd.read_csv('/Workspace/Users/barney.hodge@easyjet.com/FY25Q3DTG+100.csv')
FY25Q3_1.sort_values(by='flight_time', inplace=True)
FY25Q3_1.info()

In [0]:
FY25Q3_2 = pd.read_csv('/Workspace/Users/barney.hodge@easyjet.com/FY25Q3DTG-100.csv')
FY25Q3_2.sort_values(by='flight_time', inplace=True)
FY25Q3_2.info()

In [0]:
FY25Q4_1 = pd.read_csv('/Workspace/Users/barney.hodge@easyjet.com/FY25Q4DTG+100.csv')
FY25Q4_1.sort_values(by='flight_time', inplace=True)
FY25Q4_1.info()

In [0]:
FY25Q4_2 = pd.read_csv('/Workspace/Users/barney.hodge@easyjet.com/FY25Q4DTG-100.csv')
FY25Q4_2.sort_values(by='flight_time', inplace=True)
FY25Q4_2.info()

In [0]:
FY26Q1 = pd.read_csv('/Workspace/Users/barney.hodge@easyjet.com/FY26Q1.csv')
FY26Q1.sort_values(by='flight_time', inplace=True)
FY26Q1.info()

In [0]:
df_internal = pd.concat([FY24Q3, FY24Q4, FY25Q1, FY25Q2, FY25Q3_1, FY25Q3_2, FY25Q4_1, FY25Q4_2, FY26Q1], ignore_index=True)
df_internal.info()

In [0]:
import numpy as np
def optimize_df(df):

    start_mem = df.memory_usage(deep=True).sum() / 1024**2
    print(f"Initial memory usage: {start_mem:.2f} MB")

    for col in df.columns:

        if '_dt' in col:
            df[col] = pd.to_datetime(df[col])

        elif 'flightkey' in col or 'region' in col or 'routetype' in col:
            df[col] = df[col].astype('category')

        elif 'float' in str(df[col].dtype):
            df[col] = pd.to_numeric(df[col], downcast='float')
            
        elif 'int' in str(df[col].dtype):
            df[col] = pd.to_numeric(df[col], downcast='integer')

    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    reduction = 100 * (start_mem - end_mem) / start_mem
    print(f"Final memory usage: {end_mem:.2f} MB ({reduction:.2f}% reduction)")

    return df

optimize_df(df_internal)
df_internal.info()

In [0]:
df_internal['sectormonth'] = (df_internal['flightkey'].astype(str).str[8:14] + df_internal['flightkey'].astype(str).str[4:6])

In [0]:
df_initial = pd.merge(df_internal, df_sold['new_cluster_label'], on='sectormonth', how='left')
df_initial.head()

In [0]:
df = df_initial.drop(columns=['flight_time','route','sectormonth','total_optionality_score','combined_bp','combined_bp_DoW','base','dest','prop_from_base','prop_from_dest','time_quality_score'], axis=1)
df.info()

In [0]:
optimize_df(df)
df.info()

In [0]:
df = df[df['cumulative_sales'] >= 0]
df.info()

In [0]:
df.dropna(inplace=True)
df.info()

In [0]:
import holidays
uk_holidays = holidays.UK(years=range(2024, 2026))
holidays_df = pd.DataFrame([(date, name) for date, name in uk_holidays.items()], columns=['ds', 'holiday'])
holidays_df.sort_values(by='ds', inplace=True)
holidays_df

In [0]:
additional_holidays = pd.DataFrame([
    {'ds': '2024-04-01', 'holiday': 'Easter Monday'},
    {'ds': '2024-08-26', 'holiday': 'Summer Bank Holiday'},
    {'ds': '2025-04-21', 'holiday': 'Easter Monday'},
    {'ds': '2025-08-25', 'holiday': 'Summer Bank Holiday'},
])
holidays_df = pd.concat([holidays_df, additional_holidays], ignore_index=True)
holidays_df.drop_duplicates(inplace=True)
holidays_df['ds'] = pd.to_datetime(holidays_df['ds'])
holidays_df.sort_values(by='ds', inplace=True)
holidays_df.reset_index(drop=True, inplace=True)
holidays_df

In [0]:
df["charge_dt"] = pd.to_datetime(df["charge_dt"])
df["flight_dt"] = pd.to_datetime(df["flight_dt"])
holidays_df["ds"] = pd.to_datetime(holidays_df["ds"])

df = df.merge(holidays_df.rename(columns={'ds': 'charge_dt', 'holiday': 'charge_dt_holiday'}), how='left', on='charge_dt')
df = df.merge(holidays_df.rename(columns={'ds': 'flight_dt', 'holiday': 'flight_dt_holiday'}), how='left', on='flight_dt')
df['is_charge_date_holiday'] = df['charge_dt_holiday'].notnull().astype(int)
df['is_flight_date_holiday'] = df['flight_dt_holiday'].notnull().astype(int)
df.drop(['charge_dt_holiday', 'flight_dt_holiday'], axis=1, inplace=True)
df.head()

In [0]:
df['is_charge_date_weekday'] = (df['charge_dt'].dt.dayofweek < 5).astype(int)
df['is_flight_date_weekday'] = (df['flight_dt'].dt.dayofweek < 5).astype(int)

In [0]:
'''import numpy as np

def profile(x100, x60, x20, x00, dtg, capacity):

    x1 = (x60 - x100)/(x00 - x100)
    x2 = (x20 - x100)/(x00 - x100)
    b = np.log(np.log((1-(x1**4))/0.6)/np.log((1-(x2**4))/0.2))/np.log(x1/x2) 
    a = np.log((1-(x1**4))/0.6)/(x1**b) 

    x_norm = (dtg - x100)/(x00 - x100)
    y_val = (np.exp(-a * (x_norm**b)))*(1-(x_norm**4))

    return y_val*capacity'''

In [0]:
#df['profile_tgt'] = profile(df['x100'], df['x60'], df['x20'], df['x00'], df['dtg'], df['ty_capacity'])

In [0]:
#df['profile_tgt'] = df['prof_LF']*df['ty_lid']

In [0]:
import numpy as np
df['profile_distance_bucket'] = pd.cut(df['Z_instrument'], bins=[-np.inf, -32, -16, -8, -4, -2, 0, 2, 4, 8, 16, 32, np.inf], labels=['-32+', '-32 to -16', '-16 to -8', '-8 to -4', '-4 to -2', '-2 to 0', '0 to 2', '2 to 4', '4 to 8', '8 to 16', '16 to 32', '32+'])
df.head()

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(12,6))
sns.boxplot(data=df, x='profile_distance_bucket', y='treatment_delta')
plt.title('Treatment Delta by Profile Distance Bucket (unt_net)')
plt.xlabel('Profile Distance Bucket')
plt.ylabel('Treatment Delta')
plt.show()

In [0]:
df['control_ROS'] = df['log_sales_pre']
df['control_loadfactor'] = np.log(df['Loadfactor'] + 0.01)
df.dropna(inplace=True)
df.info()

In [0]:
df = df.drop('log_sales_pre', axis=1)

In [0]:
import statsmodels.api as sm

X_first_stage = df[['Z_instrument', 'control_ROS', 'control_loadfactor', 'dtg']]
y_first_stage = df['treatment_delta']

X_first_stage = sm.add_constant(X_first_stage)

model_first_stage = sm.OLS(y_first_stage, X_first_stage).fit()

print(model_first_stage.summary())

f_test = model_first_stage.f_test("Z_instrument = 0")
f_stat = f_test.fvalue

print(f"Instrument F-Statistic: {f_stat}")

if f_stat > 10:
    print("PASS: Instrument is Strong. (F > 10)")
else:
    print("FAIL: Instrument is Weak. (F < 10)")

In [0]:
cyclic_cols = ['flight_dow', 'charge_dow', 'flight_dom', 'charge_dom', 'flight_mth', 'charge_mth']

def encode_cyclic_features(df, cols):
    for col in cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        max_val = df[col].max()
        df[col + '_sin'] = (np.sin(2 * np.pi * df[col] / max_val)).astype('float16')
        df[col + '_cos'] = (np.cos(2 * np.pi * df[col] / max_val)).astype('float16')
        df.drop(col, axis=1, inplace=True)
    return df

df = encode_cyclic_features(df, cyclic_cols)

In [0]:
df['day_number'] = (df['charge_dt'] - pd.to_datetime('2024-04-01')).dt.days.astype(int)

In [0]:
df = df.drop('profile_distance_bucket', axis=1)
optimize_df(df)
df.info()

In [0]:
df = df.sort_values(by=['charge_dt','flight_dt'])

df_train = df[df['charge_dt'] < '2025-07-01']
df_test = df[df['charge_dt'] >= '2025-07-01']

In [0]:
df_train.tail()

In [0]:
df_test.head()

In [0]:
df_train.info()

In [0]:
df_test.info()

In [0]:
from sklearn import set_config
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


set_config(transform_output='pandas')

cols_to_scale = ['dtg','total_linear_optionality_score','combined_bp_DoW_DTG','sale_length','lag_sales_7','lag_sales_14','lag_sales_21','lag_sales_28','SF7C7','SF14C14','SF21C21','SF28C28','Loadfactor','control_loadfactor','control_ROS','sale_period_progress','new_cluster_label','lid','Z_instrument','treatment_delta','cumulative_sales','seats_remaining','day_number']

preprocessor = ColumnTransformer([('scaler', StandardScaler(), cols_to_scale)], remainder='passthrough', verbose_feature_names_out=False )

pipeline = Pipeline([('preprocessor', preprocessor)])

df_train_scaled = pipeline.fit_transform(df_train)
df_test_scaled = pipeline.transform(df_test)
#df_train_scaled['dtg_sq_scaled'] = df_train_scaled['dtg'] ** 2
#df_test_scaled['dtg_sq_scaled'] = df_test_scaled['dtg'] ** 2
df_train_scaled.head()

Select Nuissance models

In [0]:
'''import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GroupKFold, cross_val_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error

mlflow.set_experiment('/Workspace/Users/barney.hodge@easyjet.com/price-dependent-demand-forecast/Apprenticeship Project (elasticity estimate)')

def get_residuals_cv(model_instance, target, controls, groups, n_splits=3):
    cv = GroupKFold(n_splits=n_splits)
    start_time = time.time()
    predicted_baseline = cross_val_predict(model_instance, controls, target, cv=cv, groups=groups)
    end_time = time.time()
    duration = end_time - start_time
    
    residuals = target - predicted_baseline
    return residuals, predicted_baseline, duration

model_candidates = {
    'Ridge_Regression0.1': Ridge(alpha=0.01),
    'Ridge_Regression10.0': Ridge(alpha=10),
    'Lasso_Regression0.01': Lasso(alpha=0.01),
    'XGBoost1': XGBRegressor(n_estimators=750, max_depth=5, learning_rate=0.01, verbosity=0, n_jobs=4),
    'XGBoost2': XGBRegressor(n_estimators=750, max_depth=5, learning_rate=0.1, verbosity=0, n_jobs=4),
    'XGBoost3': XGBRegressor(n_estimators=750, max_depth=5, learning_rate=1, verbosity=0, n_jobs=4),
    'XGBoost4': XGBRegressor(n_estimators=750, max_depth=3, learning_rate=0.1, verbosity=0, n_jobs=4),
    'XGBoost5': XGBRegressor(n_estimators=750, max_depth=7, learning_rate=0.1, verbosity=0, n_jobs=4),
    'XGBoost6': XGBRegressor(n_estimators=750, max_depth=9, learning_rate=0.1, verbosity=0, n_jobs=4),
    'XGBoost7': XGBRegressor(n_estimators=1000, max_depth=5, learning_rate=0.1, verbosity=0, n_jobs=4),
    'XGBoost8': XGBRegressor(n_estimators=1500, max_depth=5, learning_rate=0.1, verbosity=0, n_jobs=4),
    'XGBoost9': XGBRegressor(n_estimators=2000, max_depth=5, learning_rate=0.1, verbosity=0, n_jobs=4)
}'''

In [0]:
'''Y = df_train_scaled['outcome_delta'].values
T = df_train_scaled['treatment_delta'].values
Z = df_train_scaled['Z_instrument'].values

W_names = ['flight_dow_sin','flight_dow_cos','charge_dow_sin','charge_dow_cos','flight_dom_sin','flight_dom_cos','charge_dom_sin','charge_dom_cos','flight_mth_sin','flight_mth_cos','charge_mth_sin','charge_mth_cos','dtg','lag_sales_7','lag_sales_14','lag_sales_21','lag_sales_28', 'SF7C7','SF14C14','SF21C21','SF28C28','Loadfactor','control_loadfactor','cumulative_sales','control_ROS','sale_length','sale_period_progress','new_cluster_label','lid','is_charge_date_holiday','is_flight_date_holiday','seats_remaining']
W = df_train_scaled[W_names].values
X = df_train_scaled[['dtg','total_linear_optionality_score','combined_bp_DoW_DTG','charge_dow_sin','charge_dow_cos']].values
X_names = ['dtg','total_linear_optionality_score','combined_bp_DoW_DTG','charge_dow_sin','charge_dow_cos']

targets_dict = {'Y_outcome': Y}#, 'T_treatment': T, 'Z_instrument': Z}
groups = df_train_scaled['flightkey'].values

def extract_importance(model, feature_names):

    importance_df = pd.DataFrame({'Feature': feature_names})
    
    if hasattr(model, 'feature_importances_'):
        importance_df['Importance'] = model.feature_importances_
        importance_df['Type'] = 'Tree Gain/Split'
        
    elif hasattr(model, 'coef_'):
        coefs = model.coef_
        if coefs.ndim > 1: 
            coefs = coefs[0]
        importance_df['Importance'] = coefs
        importance_df['Type'] = 'Linear Coefficient'
        
    else:
        importance_df['Importance'] = 0
        importance_df['Type'] = 'Unknown'

    importance_df['Abs_Importance'] = importance_df['Importance'].abs()
    return importance_df.sort_values(by='Abs_Importance', ascending=False)

# START EXPERIMENT LOOP
for model_name, model_inst in model_candidates.items():

    with mlflow.start_run(run_name=model_name):

        print(f'Logging run for: {model_name}...')
        mlflow.log_param('model_type', model_name)

        if hasattr(model_inst, 'get_params'):
            mlflow.log_params(model_inst.get_params())

        total_duration = 0        
        results_storage = {}

        for target_name, target_data in targets_dict.items():

            res, preds, duration = get_residuals_cv(model_inst, target_data, W, groups)
            total_duration += duration
            
            rmse = np.sqrt(mean_squared_error(target_data, preds))
            mae = mean_absolute_error(target_data, preds)
            r2 = 1 - (np.sum(res**2) / np.sum((target_data - target_data.mean())**2))
            
            mlflow.log_metric(f'{target_name}_rmse', rmse)
            mlflow.log_metric(f'{target_name}_mae', mae)
            mlflow.log_metric(f'{target_name}_r2', r2)
            
            results_storage[target_name] = res

            from sklearn.base import clone
            explain_model = clone(model_inst) 
            explain_model.fit(W, target_data)
            
            # Extract
            imp_df = extract_importance(explain_model, W_names)
            
            # Save CSV artifact (Raw numbers)
            csv_name = f"importance_{target_name}.csv"
            imp_df.to_csv(csv_name, index=False)
            mlflow.log_artifact(csv_name)
            
            # Generate Plot (Top 20 features)
            plt.figure(figsize=(10, 6))
            sns.barplot(
                data=imp_df.head(20), 
                x='Importance', 
                y='Feature', 
                palette='viridis'
            )
            plt.title(f"Top 20 Drivers for {target_name} \n({model_name})")
            plt.axvline(0, color='k', linewidth=0.8) # Zero line for coefficients
            plt.tight_layout()
            
            # Save Plot artifact
            plot_name = f"importance_plot_{target_name}.png"
            plt.savefig(plot_name)
            mlflow.log_artifact(plot_name)
            plt.close()
            

        mlflow.log_metric('total_cv_time_seconds', total_duration)
        
        # --- ARTIFACTS: RESIDUAL DISTRIBUTION ---
        # A good residual should look like Gaussian noise centered at 0.
        # If it's skewed, the model missed something.
        
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        for i, (key, val) in enumerate(results_storage.items()):
            sns.histplot(val, kde=True, ax=axes[i], bins=50)
            axes[i].set_title(f'{key} Residuhials')
            axes[i].set_xlabel('Residual Value')
        
        plt.tight_layout()
        plot_name = 'residual_distributions.png'
        plt.savefig(plot_name)
        mlflow.log_artifact(plot_name)
        plt.close(fig)
        
        print(f'Run {model_name} complete. Total CV Time: {total_duration:.2f}s')'''

In [0]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_predict, KFold, GroupKFold

Y = df_train_scaled['outcome_delta'].values
T = df_train_scaled['treatment_delta'].values
Z = df_train_scaled['Z_instrument'].values

W = df_train_scaled[['flight_dow_sin','flight_dow_cos','charge_dow_sin','charge_dow_cos','flight_dom_sin','flight_dom_cos','charge_dom_sin','charge_dom_cos','flight_mth_sin','flight_mth_cos','charge_mth_sin','charge_mth_cos','dtg','lag_sales_7','lag_sales_14','lag_sales_21','lag_sales_28', 'SF7C7','SF14C14','SF21C21','SF28C28','Loadfactor','control_loadfactor','cumulative_sales','control_ROS','sale_length','sale_period_progress','new_cluster_label','lid','is_charge_date_holiday','is_flight_date_holiday','seats_remaining','day_number']].values

X = df_train_scaled[['dtg','total_linear_optionality_score','combined_bp_DoW_DTG']].values
X_names = ['dtg','total_linear_optionality_score','combined_bp_DoW_DTG']

print(f"Running Manual DML on {len(Y)} rows...")
print("Step 1: Cleaning Data (Orthogonalization)...")

groups = df_train_scaled['flightkey'].values

def get_residuals(target, controls, groups):

    xgbm = XGBRegressor(n_estimators=750, max_depth=7,learning_rate=0.1,verbose=0)
    cv = GroupKFold(n_splits=5)
    predicted_baseline = cross_val_predict(xgbm, controls, target, cv=cv, groups=groups, n_jobs=1)
    return target - predicted_baseline

y_res = get_residuals(Y, W, groups)
t_res = get_residuals(T, W, groups)
z_res = get_residuals(Z, W, groups)

print("Step 2: Creating Interactions...")

T_interaction = t_res.reshape(-1, 1) * X
T_final = np.column_stack([t_res, T_interaction])

Z_interaction = z_res.reshape(-1, 1) * X
Z_final = np.column_stack([z_res, Z_interaction])

print("Step 3: Running Final IV Regression...")

from linearmodels.iv import IV2SLS

interaction_names = [f"Price_x_{col}" for col in X_names]
exog_names = ['Price'] + interaction_names

df_Y_res = pd.DataFrame(y_res, columns=['Sales_Res'])
df_T_final = pd.DataFrame(T_final, columns=exog_names)
df_Z_final = pd.DataFrame(Z_final, columns=[f"Instr_{i}" for i in range(Z_final.shape[1])])

model = IV2SLS(dependent=df_Y_res, exog=None, endog=df_T_final, instruments=df_Z_final)

results = model.fit()
print(results)

In [0]:
#3

BETA_INTERCEPT = -0.2413 # price is inversely proportional to sales - elasticity model (expected)
BETA_DTG = -0.0398 # when dtg increases, elasticty increases (expected)
BETA_OPT = -0.0336 # when optionality increases, elasticity increases (expected)
BETA_BP = 0.0486 # when business penetration increases, elasticity decreases (expected)

def predict_elasticity(row):
    e = BETA_INTERCEPT
    e += BETA_DTG * row['dtg']                 
    e += BETA_BP * row['combined_bp_DoW_DTG']     
    e += BETA_OPT * row['total_linear_optionality_score']
    return e

# Apply to Test Set (Make sure it's scaled!)
df_test_scaled['predicted_elasticity'] = df_test_scaled.apply(predict_elasticity, axis=1)

# ---------------------------------------------------------
# 2. CREATE BUCKETS (QUINTILES)
# ---------------------------------------------------------
# Sort by predicted elasticity and split into 5 groups
df_test_scaled['elasticity_bin'] = pd.qcut(df_test_scaled['predicted_elasticity'], 5, labels=[1, 2, 3, 4, 5])

print("Buckets created. Calculating ACTUAL elasticity per bucket...")

# ---------------------------------------------------------
# 3. VERIFY EACH BUCKET (THE TRUTH TEST)
# ---------------------------------------------------------
# We run a mini-IV regression for each bin to see the REAL slope.

bucket_results = []
bucket_labels = []

for bin_num in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    # Filter data for this bucket
    subset = df_test_scaled[df_test_scaled['elasticity_bin'] == bin_num]
    
    # Run simple IV: Sales ~ Price (Instrumented by Z)
    # We use the same 'get_residuals' logic or just raw IV if sample is large enough
    # Here we use raw IV for simplicity of the test check
    iv_mod = IV2SLS(
        dependent=subset['outcome_delta'],
        exog=None,
        endog=subset['treatment_delta'],
        instruments=subset['Z_instrument']
    ).fit()
    
    # The coefficient of 'treatment_delta' is the ACTUAL elasticity of this group
    real_elasticity = iv_mod.params['treatment_delta']
    bucket_results.append(real_elasticity)
    bucket_labels.append(f"Bin {bin_num}")
    
    print(f"Bin {bin_num} (Model pred: {subset['predicted_elasticity'].mean():.2f}) -> Actual Slope: {real_elasticity:.2f}")

# ---------------------------------------------------------
# 4. VISUALIZE THE SUCCESS
# ---------------------------------------------------------
plt.figure(figsize=(10, 6))
plt.bar(bucket_labels, bucket_results, color='skyblue', edgecolor='black')
plt.ylabel('Actual Observed Elasticity (IV Slope)')
plt.xlabel('Model Predicted Sensitivity (Bin 1 = Most Elastic)')
plt.title('Model Validation: Did we correctly sort the flights?')
plt.axhline(0, color='black', linewidth=1)
plt.show()

In [0]:
#XGB
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_predict, KFold, GroupKFold

Y = df_train_scaled['outcome_delta'].values
T = df_train_scaled['treatment_delta'].values
Z = df_train_scaled['Z_instrument'].values

W = df_train_scaled[['routetype','region','flight_dow_sin','flight_dow_cos','charge_dow_sin','charge_dow_cos','flight_dom_sin','flight_dom_cos','charge_dom_sin','charge_dom_cos','flight_mth_sin','flight_mth_cos','charge_mth_sin','charge_mth_cos','dtg','lag_sales_7','lag_sales_14','lag_sales_21','lag_sales_28', 'SF7C7','SF14C14','SF21C21','SF28C28','Loadfactor','control_loadfactor','cumulative_sales','control_ROS','sale_length','sale_period_progress','new_cluster_label','lid','is_charge_date_holiday','is_flight_date_holiday','seats_remaining','day_number']].values

X = df_train_scaled[['dtg','total_linear_optionality_score','combined_bp_DoW_DTG','charge_dow_sin','charge_dow_cos']].values
X_names = ['dtg','total_linear_optionality_score','combined_bp_DoW_DTG','charge_dow_sin','charge_dow_cos']

print(f"Running Manual DML on {len(Y)} rows...")
print("Step 1: Cleaning Data (Orthogonalization)...")

groups = df_train_scaled['flightkey'].values

def get_residuals(target, controls, groups):

    xgbm = XGBRegressor(n_estimators=750, max_depth=7,learning_rate=0.1,verbose=0)
    cv = GroupKFold(n_splits=5)
    predicted_baseline = cross_val_predict(xgbm, controls, target, cv=cv, groups=groups, n_jobs=1)
    return target - predicted_baseline

y_res = get_residuals(Y, W, groups)
t_res = get_residuals(T, W, groups)
z_res = get_residuals(Z, W, groups)

print("Step 2: Creating Interactions...")

T_interaction = t_res.reshape(-1, 1) * X
T_final = np.column_stack([t_res, T_interaction])

Z_interaction = z_res.reshape(-1, 1) * X
Z_final = np.column_stack([z_res, Z_interaction])

print("Step 3: Running Final IV Regression...")

from linearmodels.iv import IV2SLS

interaction_names = [f"Price_x_{col}" for col in X_names]
exog_names = ['Price'] + interaction_names

df_Y_res = pd.DataFrame(y_res, columns=['Sales_Res'])
df_T_final = pd.DataFrame(T_final, columns=exog_names)
df_Z_final = pd.DataFrame(Z_final, columns=[f"Instr_{i}" for i in range(Z_final.shape[1])])

model = IV2SLS(dependent=df_Y_res, exog=None, endog=df_T_final, instruments=df_Z_final)

results = model.fit()
print(results)

In [0]:
#3

BETA_INTERCEPT = -0.3196 # price is inversely proportional to sales - elasticity model (expected)
BETA_DTG = -0.0462 # when dtg increases, elasticty increases (expected)
BETA_OPT = -0.0128 # when optionality increases, elasticity increases (expected)
BETA_BP = 0.0285 # when business penetration increases, elasticity decreases (expected)
BETA_CDOW_SIN = 0.0266
BETA_CDOW_COS = -0.0425



def predict_elasticity(row):
    e = BETA_INTERCEPT
    e += BETA_DTG * row['dtg']                 
    e += BETA_BP * row['combined_bp_DoW_DTG']     
    e += BETA_OPT * row['total_linear_optionality_score']
    e += BETA_CDOW_SIN * row['charge_dow_sin']
    e += BETA_CDOW_COS * row['charge_dow_cos'] 
    return e

# Apply to Test Set (Make sure it's scaled!)
df_test_scaled['predicted_elasticity'] = df_test_scaled.apply(predict_elasticity, axis=1)

# ---------------------------------------------------------
# 2. CREATE BUCKETS (QUINTILES)
# ---------------------------------------------------------
# Sort by predicted elasticity and split into 5 groups
df_test_scaled['elasticity_bin'] = pd.qcut(df_test_scaled['predicted_elasticity'], 5, labels=[1, 2, 3, 4, 5])

print("Buckets created. Calculating ACTUAL elasticity per bucket...")

# ---------------------------------------------------------
# 3. VERIFY EACH BUCKET (THE TRUTH TEST)
# ---------------------------------------------------------
# We run a mini-IV regression for each bin to see the REAL slope.

bucket_results = []
bucket_labels = []

for bin_num in [1, 2, 3, 4, 5]:
    # Filter data for this bucket
    subset = df_test_scaled[df_test_scaled['elasticity_bin'] == bin_num]
    
    # Run simple IV: Sales ~ Price (Instrumented by Z)
    # We use the same 'get_residuals' logic or just raw IV if sample is large enough
    # Here we use raw IV for simplicity of the test check
    iv_mod = IV2SLS(
        dependent=subset['outcome_delta'],
        exog=None,
        endog=subset['treatment_delta'],
        instruments=subset['Z_instrument']
    ).fit()
    
    # The coefficient of 'treatment_delta' is the ACTUAL elasticity of this group
    real_elasticity = iv_mod.params['treatment_delta']
    bucket_results.append(real_elasticity)
    bucket_labels.append(f"Bin {bin_num}")
    
    print(f"Bin {bin_num} (Model pred: {subset['predicted_elasticity'].mean():.2f}) -> Actual Slope: {real_elasticity:.2f}")

# ---------------------------------------------------------
# 4. VISUALIZE THE SUCCESS
# ---------------------------------------------------------
plt.figure(figsize=(10, 6))
plt.bar(bucket_labels, bucket_results, color='skyblue', edgecolor='black')
plt.ylabel('Actual Observed Elasticity (IV Slope)')
plt.xlabel('Model Predicted Sensitivity (Bin 1 = Most Elastic)')
plt.title('Model Validation: Did we correctly sort the flights?')
plt.axhline(0, color='black', linewidth=1)
plt.show()

In [0]:
#XGB
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_predict, KFold, GroupKFold

Y = df_train_scaled['outcome_delta'].values
T = df_train_scaled['treatment_delta'].values
Z = df_train_scaled['Z_instrument'].values

W = df_train_scaled[['region','routetype','flight_dow_sin','flight_dow_cos','charge_dow_sin','charge_dow_cos','flight_dom_sin','flight_dom_cos','charge_dom_sin','charge_dom_cos','flight_mth_sin','flight_mth_cos','charge_mth_sin','charge_mth_cos','dtg','lag_sales_7','lag_sales_14','lag_sales_21','lag_sales_28', 'SF7C7','SF14C14','SF21C21','SF28C28','Loadfactor','control_loadfactor','cumulative_sales','control_ROS','sale_length','sale_period_progress','new_cluster_label','lid','is_charge_date_holiday','is_flight_date_holiday','seats_remaining']].values

X = df_train_scaled[['dtg','total_linear_optionality_score','combined_bp_DoW_DTG','charge_dow_sin','charge_dow_cos','charge_dom_sin','charge_dom_cos']].values
X_names = ['dtg','total_linear_optionality_score','combined_bp_DoW_DTG','charge_dow_sin','charge_dow_cos','charge_dom_sin','charge_dom_cos']

print(f"Running Manual DML on {len(Y)} rows...")
print("Step 1: Cleaning Data (Orthogonalization)...")

groups = df_train_scaled['flightkey'].values

def get_residuals(target, controls, groups):

    xgbm = XGBRegressor(n_estimators=1000, max_depth=5,learning_rate=0.1,verbose=-1)
    cv = GroupKFold(n_splits=3)
    predicted_baseline = cross_val_predict(xgbm, controls, target, cv=cv, groups=groups, n_jobs=1)
    return target - predicted_baseline

y_res = get_residuals(Y, W, groups)
t_res = get_residuals(T, W, groups)
z_res = get_residuals(Z, W, groups)

print("Step 2: Creating Interactions...")

T_interaction = t_res.reshape(-1, 1) * X
T_final = np.column_stack([t_res, T_interaction])

Z_interaction = z_res.reshape(-1, 1) * X
Z_final = np.column_stack([z_res, Z_interaction])

print("Step 3: Running Final IV Regression...")

from linearmodels.iv import IV2SLS

interaction_names = [f"Price_x_{col}" for col in X_names]
exog_names = ['Price'] + interaction_names

df_Y_res = pd.DataFrame(y_res, columns=['Sales_Res'])
df_T_final = pd.DataFrame(T_final, columns=exog_names)
df_Z_final = pd.DataFrame(Z_final, columns=[f"Instr_{i}" for i in range(Z_final.shape[1])])

model = IV2SLS(dependent=df_Y_res, exog=None, endog=df_T_final, instruments=df_Z_final)

results = model.fit()
print(results)

In [0]:
BETA_INTERCEPT = -0.3216 # price is inversely proportional to sales - elasticity model (expected)
BETA_DTG = -0.0460 # when dtg increases, elasticty increases (expected)
BETA_OPT = -0.0134 # when optionality increases, elasticity increases (expected)
BETA_BP = 0.0280 # when business penetration increases, elasticity decreases (expected)
BETA_CDOW_SIN = 0.0268
BETA_CDOW_COS = -0.0425
BETA_CDOM_SIN = 0.0251
BETA_CDOM_COS = -0.0145



def predict_elasticity(row):
    e = BETA_INTERCEPT
    e += BETA_DTG * row['dtg']                 
    e += BETA_BP * row['combined_bp_DoW_DTG']     
    e += BETA_OPT * row['total_linear_optionality_score']
    e += BETA_CDOW_SIN * row['charge_dow_sin']
    e += BETA_CDOW_COS * row['charge_dow_cos']
    e += BETA_CDOM_SIN * row['charge_dom_sin']
    e += BETA_CDOM_COS * row['charge_dom_cos']
    return e

# Apply to Test Set (Make sure it's scaled!)
df_test_scaled['predicted_elasticity'] = df_test_scaled.apply(predict_elasticity, axis=1)

# ---------------------------------------------------------
# 2. CREATE BUCKETS (QUINTILES)
# ---------------------------------------------------------
# Sort by predicted elasticity and split into 5 groups
df_test_scaled['elasticity_bin'] = pd.qcut(df_test_scaled['predicted_elasticity'], 5, labels=[1, 2, 3, 4, 5])

print("Buckets created. Calculating ACTUAL elasticity per bucket...")

# ---------------------------------------------------------
# 3. VERIFY EACH BUCKET (THE TRUTH TEST)
# ---------------------------------------------------------
# We run a mini-IV regression for each bin to see the REAL slope.

bucket_results = []
bucket_labels = []

for bin_num in [1, 2, 3, 4, 5]:
    # Filter data for this bucket
    subset = df_test_scaled[df_test_scaled['elasticity_bin'] == bin_num]
    
    # Run simple IV: Sales ~ Price (Instrumented by Z)
    # We use the same 'get_residuals' logic or just raw IV if sample is large enough
    # Here we use raw IV for simplicity of the test check
    iv_mod = IV2SLS(
        dependent=subset['outcome_delta'],
        exog=None,
        endog=subset['treatment_delta'],
        instruments=subset['Z_instrument']
    ).fit()
    
    # The coefficient of 'treatment_delta' is the ACTUAL elasticity of this group
    real_elasticity = iv_mod.params['treatment_delta']
    bucket_results.append(real_elasticity)
    bucket_labels.append(f"Bin {bin_num}")
    
    print(f"Bin {bin_num} (Model pred: {subset['predicted_elasticity'].mean():.2f}) -> Actual Slope: {real_elasticity:.2f}")

# ---------------------------------------------------------
# 4. VISUALIZE THE SUCCESS
# ---------------------------------------------------------
plt.figure(figsize=(10, 6))
plt.bar(bucket_labels, bucket_results, color='skyblue', edgecolor='black')
plt.ylabel('Actual Observed Elasticity (IV Slope)')
plt.xlabel('Model Predicted Sensitivity (Bin 1 = Most Elastic)')
plt.title('Model Validation: Did we correctly sort the flights?')
plt.axhline(0, color='black', linewidth=1)
plt.show()

In [0]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_predict, KFold, GroupKFold

Y = df_train_scaled['outcome_delta'].values
T = df_train_scaled['treatment_delta'].values
Z = df_train_scaled['Z_instrument'].values

W = df_train_scaled[['flight_dow_sin','flight_dow_cos','charge_dow_sin','charge_dow_cos','flight_dom_sin','flight_dom_cos','charge_dom_sin','charge_dom_cos','flight_mth_sin','flight_mth_cos','charge_mth_sin','charge_mth_cos','dtg','lag_sales_7','lag_sales_14','lag_sales_21','lag_sales_28', 'SF7C7','SF14C14','SF21C21','SF28C28','Loadfactor','control_loadfactor','cumulative_sales','control_ROS','sale_length','sale_period_progress','new_cluster_label','lid','is_charge_date_holiday','is_flight_date_holiday','seats_remaining']].values

X = df_train_scaled[['dtg','total_linear_optionality_score','combined_bp_DoW_DTG','charge_dow_sin','charge_dow_cos']].values
X_names = ['dtg','total_linear_optionality_score','combined_bp_DoW_DTG','charge_dow_sin','charge_dow_cos']

print(f"Running Manual DML on {len(Y)} rows...")
print("Step 1: Cleaning Data (Orthogonalization)...")

groups = df_train_scaled['flightkey'].values

def get_residuals(target, controls, groups):

    xgbm = XGBRegressor(n_estimators=2000, max_depth=7,learning_rate=0.1,verbose=-1)
    cv = GroupKFold(n_splits=5)
    predicted_baseline = cross_val_predict(xgbm, controls, target, cv=cv, groups=groups, n_jobs=1)
    return target - predicted_baseline

y_res = get_residuals(Y, W, groups)
t_res = get_residuals(T, W, groups)
z_res = get_residuals(Z, W, groups)

print("Step 2: Creating Interactions...")

T_interaction = t_res.reshape(-1, 1) * X
T_final = np.column_stack([t_res, T_interaction])

Z_interaction = z_res.reshape(-1, 1) * X
Z_final = np.column_stack([z_res, Z_interaction])

print("Step 3: Running Final IV Regression...")

from linearmodels.iv import IV2SLS

interaction_names = [f"Price_x_{col}" for col in X_names]
exog_names = ['Price'] + interaction_names

df_Y_res = pd.DataFrame(y_res, columns=['Sales_Res'])
df_T_final = pd.DataFrame(T_final, columns=exog_names)
df_Z_final = pd.DataFrame(Z_final, columns=[f"Instr_{i}" for i in range(Z_final.shape[1])])

model = IV2SLS(dependent=df_Y_res, exog=None, endog=df_T_final, instruments=df_Z_final)

results = model.fit()
print(results)

In [0]:
BETA_INTERCEPT = -0.3601 # price is inversely proportional to sales - elasticity model (expected)
BETA_DTG = -0.1126 # when dtg increases, elasticty increases (expected)
BETA_OPT = -0.0297 # when optionality increases, elasticity increases (expected)
BETA_BP = 0.0301 # when business penetration increases, elasticity decreases (expected)
BETA_CDOW_SIN = 0.0346
BETA_CDOW_COS = -0.0323


def predict_elasticity(row):
    e = BETA_INTERCEPT
    e += BETA_DTG * row['dtg']                 
    e += BETA_BP * row['combined_bp_DoW_DTG']     
    e += BETA_OPT * row['total_linear_optionality_score']
    e += BETA_CDOW_SIN * row['charge_dow_sin']
    e += BETA_CDOW_COS * row['charge_dow_cos'] 
    return e

# Apply to Test Set (Make sure it's scaled!)
df_test_scaled['predicted_elasticity'] = df_test_scaled.apply(predict_elasticity, axis=1)

# ---------------------------------------------------------
# 2. CREATE BUCKETS (QUINTILES)
# ---------------------------------------------------------
# Sort by predicted elasticity and split into 5 groups
df_test_scaled['elasticity_bin'] = pd.qcut(df_test_scaled['predicted_elasticity'], 5, labels=[1, 2, 3, 4, 5])

print("Buckets created. Calculating ACTUAL elasticity per bucket...")

# ---------------------------------------------------------
# 3. VERIFY EACH BUCKET (THE TRUTH TEST)
# ---------------------------------------------------------
# We run a mini-IV regression for each bin to see the REAL slope.

bucket_results = []
bucket_labels = []

for bin_num in [1, 2, 3, 4, 5]:
    # Filter data for this bucket
    subset = df_test_scaled[df_test_scaled['elasticity_bin'] == bin_num]
    
    # Run simple IV: Sales ~ Price (Instrumented by Z)
    # We use the same 'get_residuals' logic or just raw IV if sample is large enough
    # Here we use raw IV for simplicity of the test check
    iv_mod = IV2SLS(
        dependent=subset['outcome_delta'],
        exog=None,
        endog=subset['treatment_delta'],
        instruments=subset['Z_instrument']
    ).fit()
    
    # The coefficient of 'treatment_delta' is the ACTUAL elasticity of this group
    real_elasticity = iv_mod.params['treatment_delta']
    bucket_results.append(real_elasticity)
    bucket_labels.append(f"Bin {bin_num}")
    
    print(f"Bin {bin_num} (Model pred: {subset['predicted_elasticity'].mean():.2f}) -> Actual Slope: {real_elasticity:.2f}")

# ---------------------------------------------------------
# 4. VISUALIZE THE SUCCESS
# ---------------------------------------------------------
plt.figure(figsize=(10, 6))
plt.bar(bucket_labels, bucket_results, color='skyblue', edgecolor='black')
plt.ylabel('Actual Observed Elasticity (IV Slope)')
plt.xlabel('Model Predicted Sensitivity (Bin 1 = Most Elastic)')
plt.title('Model Validation: Did we correctly sort the flights?')
plt.axhline(0, color='black', linewidth=1)
plt.show()